## Análise Descritiva e Preditiva dos dados relativos ao Sistema de Operações do Crédito Rural e do Proagro (Sicor)

###  ADENILSON SILVA

## EXTRAÇÃO, TRANSFORMAÇÃO E CARRAGAMENTO DE DADOS

###  1 - Importando bibliotecas e criando funções

In [1]:
import pandas as pd  # Manipulação de dados tabulares (DataFrame)
import numpy as np  # Operações com arrays e funções matemáticas
import matplotlib.pyplot as plt  # Criação de gráficos
import seaborn as sns
import basedosdados as bd  # Acesso a dados públicos da Base dos Dados
from datetime import datetime  # Manipulação de datas e horas

In [2]:
def categoria_tempo_empresa(idade):
    if idade <= 5:
        return 'Nova'
    elif idade <= 15:
        return 'Em crescimento'
    elif idade <= 30:
        return 'Consolidada'
    else:
        return 'Tradicional'

### 2 - Leitura dos dados

In [3]:
billing_id = "projeto"

In [4]:
%%time

query = """
 SELECT
    empreendimento.id_empreendimento as id_empreendimento,
    empreendimento.data_inicio as data_inicio_empreendimento,
    empreendimento.data_fim as data_fim_emprendimento,
    empreendimento.finalidade as finalidade,
    empreendimento.atividade as atividade,
    empreendimento.modalidade as modalidade,
    empreendimento.produto as produto,
    empreendimento.variedade as variedade,
    empreendimento.cesta_safra as cesta_safra,
    empreendimento.zoneamento as zoneamento,
    empreendimento.consorcio as consorcio
FROM `basedosdados.br_bcb_sicor.empreendimento` AS empreendimento 
"""

#Salvando dados brutos oriundos do Google Cloud
data = bd.read_sql(query = query, billing_project_id = billing_id) 
data.to_parquet('dados/dados_brutos/dados_brutos_empreendimento.parquet')
df_empreendimento = pd.read_parquet('dados/dados_brutos/dados_brutos_empreendimento.parquet')
df_empreendimento.head()

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|
Wall time: 2 s


,id_empreendimento,data_inicio_empreendimento,data_fim_emprendimento,finalidade,atividade,modalidade,produto,variedade,cesta_safra,zoneamento,consorcio
0,22507460333408,2018-08-29,NaT,custeio,pecuario(a),pesca,armacao para barco de pesca,manutencao/aquisicao,ano civil / ano de exploracao,None,valor invalido
1,12016565000053,2021-02-03,NaT,custeio,agricola,lavoura,sapoti,nao se aplica,irrigadas,None,nao consorciado
2,12016560000043,2017-05-15,NaT,custeio,agricola,lavoura,salsa,nao se aplica,fruticultura / olericultura,None,nao consorciado
3,12014580000013,2017-03-20,2020-04-03,custeio,agricola,lavoura,linho,nao se aplica,safra de verao (1ª safra),None,nao consorciado
4,12014320000043,2017-03-20,NaT,custeio,agricola,lavoura,lichia (lechia),nao se aplica,fruticultura / olericultura,None,nao consorciado


In [5]:
%%time

query = """
 SELECT id_empreendimento as id_empreendimento, 
 ano as ano, 
 taxa_juro as taxa_juro,
 sigla_uf as sigla_uf
FROM `basedosdados.br_bcb_sicor.microdados_operacao`
 WHERE id_empreendimento IS NOT NULL
"""

#Salvando dados brutos oriundos do Google Cloud
data = bd.read_sql(query = query, billing_project_id = billing_id) 
data.to_parquet('dados/dados_brutos/dados_brutos_operacao.parquet')
df_operacao = pd.read_parquet('dados/dados_brutos/dados_brutos_operacao.parquet')
df_operacao.head()

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|
Wall time: 21min 50s


,id_empreendimento,ano,taxa_juro,sigla_uf
0,23271300191002,2016,0.5,AL
1,23271300191002,2016,0.5,AL
2,23271300193002,2016,5.5,AL
3,23271300193002,2016,5.5,BA
4,23271300193002,2016,0.5,AL


In [6]:
# Junção dos dataframes
df = pd.merge(df_empreendimento, df_operacao, how='inner', on='id_empreendimento')
df.head()

,id_empreendimento,data_inicio_empreendimento,data_fim_emprendimento,finalidade,atividade,modalidade,produto,variedade,cesta_safra,zoneamento,consorcio,ano,taxa_juro,sigla_uf
0,22507460333408,2018-08-29,NaT,custeio,pecuario(a),pesca,armacao para barco de pesca,manutencao/aquisicao,ano civil / ano de exploracao,None,valor invalido,2020,6.00,SC
1,22507460333408,2018-08-29,NaT,custeio,pecuario(a),pesca,armacao para barco de pesca,manutencao/aquisicao,ano civil / ano de exploracao,None,valor invalido,2023,5.00,SP
2,22507460333408,2018-08-29,NaT,custeio,pecuario(a),pesca,armacao para barco de pesca,manutencao/aquisicao,ano civil / ano de exploracao,None,valor invalido,2023,5.00,SC
3,22507460333408,2018-08-29,NaT,custeio,pecuario(a),pesca,armacao para barco de pesca,manutencao/aquisicao,ano civil / ano de exploracao,None,valor invalido,2022,4.50,ES
4,22507460333408,2018-08-29,NaT,custeio,pecuario(a),pesca,armacao para barco de pesca,manutencao/aquisicao,ano civil / ano de exploracao,None,valor invalido,2021,5.75,MA


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21497976 entries, 0 to 21497975
Data columns (total 14 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   id_empreendimento           object 
 1   data_inicio_empreendimento  dbdate 
 2   data_fim_emprendimento      dbdate 
 3   finalidade                  object 
 4   atividade                   object 
 5   modalidade                  object 
 6   produto                     object 
 7   variedade                   object 
 8   cesta_safra                 object 
 9   zoneamento                  object 
 10  consorcio                   object 
 11  ano                         Int64  
 12  taxa_juro                   float64
 13  sigla_uf                    object 
dtypes: Int64(1), dbdate(2), float64(1), object(10)
memory usage: 2.3+ GB


In [8]:
df.describe(include='all')

,id_empreendimento,data_inicio_empreendimento,data_fim_emprendimento,finalidade,atividade,modalidade,produto,variedade,cesta_safra,zoneamento,consorcio,ano,taxa_juro,sigla_uf
count,21497976,21497976,7167034,21490576,21497976,21497976,21497976,21497976,21497976,19254698,21404452,21497976.0,2.149798e+07,21497976
unique,2327,396,104,3,2,51,375,321,125,90,53,<NA>,NaN,27
top,12016720000011,1984-01-01,2020-02-29,investimento,pecuario(a),lavoura,bovinos,nao se aplica,ano civil / ano de exploracao,nao zoneado,nao consorciado,<NA>,NaN,RS
freq,1564283,12068783,2072751,11851916,11662546,6002941,5985014,8191510,10443000,9724233,14351582,<NA>,NaN,3473579
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017.283651,3.619639e+00,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.026584,3.478915e+00,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013.0,-2.520000e+00,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.0,5.000000e-01,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017.0,3.000000e+00,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020.0,5.500000e+00,NaN


### 3 - Tratando os dados

In [9]:
# Convertendo o tipo da coluna datas e criando a coluna 'ano_criacao_empresa'
df['data_inicio_empreendimento'] = pd.to_datetime(df['data_inicio_empreendimento']) 
df['data_fim_emprendimento'] = pd.to_datetime(df['data_fim_emprendimento'])
df['ano_criacao_empresa'] = df['data_inicio_empreendimento'].dt.year

In [10]:
# Criando a coluna 'categoria_empresa' que define se a empresa é Nova, Em crescimento, Consolidada ou Tradicional
ano_atual = datetime.now().year
df['categoria_empresa'] = (ano_atual - df['ano_criacao_empresa']).apply(categoria_tempo_empresa)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21497976 entries, 0 to 21497975
Data columns (total 16 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   id_empreendimento           object        
 1   data_inicio_empreendimento  datetime64[ns]
 2   data_fim_emprendimento      datetime64[ns]
 3   finalidade                  object        
 4   atividade                   object        
 5   modalidade                  object        
 6   produto                     object        
 7   variedade                   object        
 8   cesta_safra                 object        
 9   zoneamento                  object        
 10  consorcio                   object        
 11  ano                         Int64         
 12  taxa_juro                   float64       
 13  sigla_uf                    object        
 14  ano_criacao_empresa         int32         
 15  categoria_empresa           object        
dtypes: Int64(1), dat

In [12]:
df.isna().sum()

id_empreendimento                    0
data_inicio_empreendimento           0
data_fim_emprendimento        14330942
finalidade                        7400
atividade                            0
modalidade                           0
produto                              0
variedade                            0
cesta_safra                          0
zoneamento                     2243278
consorcio                        93524
ano                                  0
taxa_juro                            0
sigla_uf                             0
ano_criacao_empresa                  0
categoria_empresa                    0
dtype: int64

In [13]:
df['finalidade'] = df['finalidade'].fillna('Outros')

In [14]:
df.isna().sum()

id_empreendimento                    0
data_inicio_empreendimento           0
data_fim_emprendimento        14330942
finalidade                           0
atividade                            0
modalidade                           0
produto                              0
variedade                            0
cesta_safra                          0
zoneamento                     2243278
consorcio                        93524
ano                                  0
taxa_juro                            0
sigla_uf                             0
ano_criacao_empresa                  0
categoria_empresa                    0
dtype: int64

In [15]:
# Removendo colunas desnecessárias
colunas = ['id_empreendimento',
            'zoneamento']
df = df.drop(columns=colunas)
df.head(5)

,data_inicio_empreendimento,data_fim_emprendimento,finalidade,atividade,modalidade,produto,variedade,cesta_safra,consorcio,ano,taxa_juro,sigla_uf,ano_criacao_empresa,categoria_empresa
0,2018-08-29,NaT,custeio,pecuario(a),pesca,armacao para barco de pesca,manutencao/aquisicao,ano civil / ano de exploracao,valor invalido,2020,6.00,SC,2018,Em crescimento
1,2018-08-29,NaT,custeio,pecuario(a),pesca,armacao para barco de pesca,manutencao/aquisicao,ano civil / ano de exploracao,valor invalido,2023,5.00,SP,2018,Em crescimento
2,2018-08-29,NaT,custeio,pecuario(a),pesca,armacao para barco de pesca,manutencao/aquisicao,ano civil / ano de exploracao,valor invalido,2023,5.00,SC,2018,Em crescimento
3,2018-08-29,NaT,custeio,pecuario(a),pesca,armacao para barco de pesca,manutencao/aquisicao,ano civil / ano de exploracao,valor invalido,2022,4.50,ES,2018,Em crescimento
4,2018-08-29,NaT,custeio,pecuario(a),pesca,armacao para barco de pesca,manutencao/aquisicao,ano civil / ano de exploracao,valor invalido,2021,5.75,MA,2018,Em crescimento


### 4 - Salvando os dados tratados

In [16]:
df.to_parquet('dados/dados_tratados/dados_tratados.parquet')
df.to_csv('dados/dados_tratados/dados_tratados.csv', index=False)

### Dados sobre versão

In [17]:
import sys
import pandas as pd
import numpy as np
import matplotlib as mat
import basedosdados as bd
print("Python:", sys.version)
print("pandas:", pd.__version__)
print("numpy:", np.__version__)
print("matplotlib:", mat.__version__)
print("basedosdados:", bd.__version__)

Python: 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
pandas: 2.2.3
numpy: 1.23.5
matplotlib: 3.9.4
basedosdados: 2.0.2
